#Install Server Requirements
This should be run on a minimum of a T4 runtime, though it will run on a CPU only session, however long TTS generations may time out/error.

This is a **work in progress**. Known issues:

- The 1st TTS generation has a brief stutter.
- RVC is not yet working.
- Transcoding/ffmpeg isnt working.
- Things yet to do on selecting your first model and other configuration setups.

If you enable DeepSpeed for XTTS models, DeepSpeed has to compile on the 1st TTS generation which can take about 90 seconds. After that it should be fine.

In [ ]:
#@markdown Click the `Play button` to the left of this message to install the requirements.<br><br>
#@markdown **OPTIONAL** Mounting your Google Drive allows you to drag and drop samples/models via the `drive/Mydrive` path. This allows you<br>
#@markdown to store or use specific audio samples or finetuned models.<br>
#@markdown **Audio samples** need to be placed in `alltalk_tts/voices`<br>
#@markdown **XTTS models** need to be placed in `alltalk_tts/models/xtts/{yourmodelfolderhere}`<br>
mount_gdrive = True #@param{type:"boolean"}

if mount_gdrive:
  from google.colab import drive
  drive.mount('/content/drive')
from IPython.display import clear_output
print("*******************************************************************")
print("** Installing server requirements. This will take 5-10 minutes ****")
print("*******************************************************************")
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 libaio-dev espeak-ng > '/dev/null' 2>&1
clear_output()
print("************************")
print("*** Cloning AllTalk ****")
print("************************\n")
!git clone -b alltalkbeta https://github.com/erew123/alltalk_tts
clear_output()
print("\n********************************")
print("*** Installing Requirements ****")
print("********************************\n")
!python -m pip install --upgrade "pip<24.1"
!pip install --quiet -r /content/alltalk_tts/system/requirements/requirements_colab.txt
clear_output()
print("\n*****************************")
print("*** Installing DeepSpeed ****")
print("*****************************\n")
!pip install deepspeed
!pip install orjson
!pip install faiss-cpu
!pip install fairseq
clear_output()
print("\n******************************")
print("*** Installing Cloudflare ****")
print("******************************\n")
# Install cloudflare
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb > '/dev/null' 2>&1
!apt install ./cloudflared-linux-amd64.deb aria2 > '/dev/null' 2>&1
!rm cloudflared-linux-amd64.deb > '/dev/null' 2>&1
!python -m spacy download en_core_web_md
clear_output()
print("\n***************************")
print("*** Installing Cutlass ****")
print("***************************\n")
# Clone the CUTLASS repository
!git clone https://github.com/NVIDIA/cutlass.git
!export CUTLASSPATH=/content/cutlass
!sudo curl -L https://github.com/BtbN/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz -o /usr/local/bin/ffmpeg.tar.xz
clear_output()
%cd /usr/local/bin/
clear_output()
!7z e /usr/local/bin/ffmpeg.tar.xz -y
clear_output()
!7z e /usr/local/bin/ffmpeg.tar -y
clear_output()
!sudo chmod a+rx /usr/local/bin/ffmpeg
clear_output()
!pip uninstall jax -y
clear_output()
print("************************************")
print("** Server requirements installed ***")
print("*** Please proceed to next step ****")
print("************************************\n")


*****************************
*** Installing DeepSpeed ****
*****************************

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.16.4-py3-none-any.whl size=1562653 sha256=298ef101f53b90d6cde2ecc7da9e326bd4478932e976dcaaec62750e6e75dc99
  Stored in directory: /root/.cache/pip/wheels/b3/6c/e5/ccad75c8ade9cb21e74721affd6d17820b1806249aac34f7f0
Successfully built deepspeed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 77.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done



# Start AllTalk TTS Server

This will start the AllTalk API and Gradio Web interface. From here you can download models, generate TTS and use external applications via the API address.

The AllTalk API address is what you would use in Kobold, SillyTavern, TGWUI's Remote extension etc if you want to generate TTS with those applications.

In [ ]:
#@markdown Click the `Play button` to the left of this message to start AllTalk API and Gradio Interface<br>
import re
import time
import json
import threading
import time

def keep_alive():
    while True:
        time.sleep(60)  # Run every 60 seconds (adjust as needed)

keep_alive_thread = threading.Thread(target=keep_alive)
keep_alive_thread.start()

Tunnel = "cloudflare"
host = "127.0.0.1"
ports = [7851, 7852]
tunnel_urls = []

# Starting tunnels for each port.
for port in ports:
    if Tunnel == "cloudflare":
        !nohup cloudflared tunnel --url http://{host}:{port} > lt_{port}.log 2>&1 &
    else:
        !nohup npx lt -p {port} > lt_{port}.log 2>&1 &

# Wait a couple of seconds for the tunnels to initialize.
time.sleep(10)

# Extract URLs for each tunnel.
for port in ports:
    log_file = f'lt_{port}.log'
    with open(log_file, 'r') as testwritefile:
        log_content = testwritefile.read()

        # Use regular expressions to find the URL.
        if Tunnel == "cloudflare":
            url_match = re.search(r'(https://[-a-z0-9]+\.trycloudflare\.com)', log_content)
        else:
            url_match = re.search(r'your url is: (https?://\S+)', log_content)

        if url_match:
            tunnel_url = url_match.group(1)
            tunnel_urls.append(tunnel_url)
        else:
            print(f"URL for port {port} not found.")

# Save the tunnel URLs to a JSON file.
try:
    # Try to open the JSON file for reading.
    with open('/content/alltalk_tts/googlecolab.json', 'r') as f:
        data = json.load(f)
except FileNotFoundError:
    # If the file doesn't exist, create an empty dictionary.
    data = {}

data['google_ip_address'] = tunnel_urls

# Write the modified data (or newly created data) back to the file.
with open('/content/alltalk_tts/googlecolab.json', 'w') as f:
    json.dump(data, f)

host = "0.0.0.0"

if Tunnel == "localtunnel":
    print("Before you copy the link above click on it and copy that ip:")
    !curl ipv4.icanhazip.com

# Start API server.
!python /content/alltalk_tts/script.py


# Start XTTS model Finetuning

Starts the Finetuning application for XTTS models.

You can either run the `Start AllTalk` to download the base XTTS model(s) for finetuning. Or you can use the folder icon on the left hand side of the screen and upload an XTTS model that you want to finetune. If you are manually uploading a model, you would place your model files in `models/xtts/{yourmodelfolderhere}` and you will need all the models files in that folder `config.json, dvae.pth, mel_stats.pth, model.pth, speakers_xtts.pth, vocab.json`. Without 1x model available, Finetuning will not start.

Likewise you can download your finetuned model from there, OR copy it to your Google Drive after finetuning, for later use in AllTalk. To access the Finetuning Gradio interface, connect to the `Google Colab Finetuning url` when Finetuning has started.

In [ ]:
#@markdown Click the `Play button` to the left of this message to start Finetuning<br>
import re
import time
import json
import threading
import time

def keep_alive():
    while True:
        time.sleep(60)  # Run every 60 seconds (adjust as needed)

keep_alive_thread = threading.Thread(target=keep_alive)
keep_alive_thread.start()

Tunnel = "cloudflare"
host = "127.0.0.1"
ports = [7052]
tunnel_urls = []

# Starting tunnels for each port.
for port in ports:
    if Tunnel == "cloudflare":
        !nohup cloudflared tunnel --url http://{host}:{port} > lt_{port}.log 2>&1 &
    else:
        !nohup npx lt -p {port} > lt_{port}.log 2>&1 &

# Wait a couple of seconds for the tunnels to initialize.
time.sleep(10)

# Extract URLs for each tunnel.
for port in ports:
    log_file = f'lt_{port}.log'
    with open(log_file, 'r') as testwritefile:
        log_content = testwritefile.read()

        # Use regular expressions to find the URL.
        if Tunnel == "cloudflare":
            url_match = re.search(r'(https://[-a-z0-9]+\.trycloudflare\.com)', log_content)
        else:
            url_match = re.search(r'your url is: (https?://\S+)', log_content)

        if url_match:
            tunnel_url = url_match.group(1)
            tunnel_urls.append(tunnel_url)
            print(f"Google Colab Finetuning url: {tunnel_url}\n")
            print(f"********************************************************************")
            print(f"**** Use the above URL to connect to Finetuning on Google Colab ****")
            print(f"********************************************************************")
            print(f"************* Now starting the Finetuning Application **************")
            print(f"********************************************************************\n")
        else:
            print(f"URL for port {port} not found.")

# Save the tunnel URLs to a JSON file.
try:
    # Try to open the JSON file for reading.
    with open('/content/alltalk_tts/googlecolab.json', 'r') as f:
        data = json.load(f)
except FileNotFoundError:
    # If the file doesn't exist, create an empty dictionary.
    data = {}

data['google_ip_address'] = tunnel_urls

# Write the modified data (or newly created data) back to the file.
with open('/content/alltalk_tts/googlecolab.json', 'w') as f:
    json.dump(data, f)

host = "0.0.0.0"

if Tunnel == "localtunnel":
    print("Before you copy the link above click on it and copy that ip:")
    !curl ipv4.icanhazip.com

# Start API server.
!python /content/alltalk_tts/finetune.py
